In [15]:
from PIL import Image
import numpy as np
import os, sys
import glob
import matplotlib.pyplot as plt
from src import data_utils

In [16]:
image_paths = glob.glob(os.path.join("data", "RPE-vscans","Original2", "*", "*.png"))
mask1_paths = glob.glob(os.path.join("data", "RPE-vscans","Masks_RPE", "*", "*.png"))
mask2_paths = glob.glob(os.path.join("data", "RPE-vscans","Masks_Retina", "*", "*.png"))

image_paths = sorted(image_paths)
mask1_paths = sorted(mask1_paths)
mask2_paths = sorted(mask2_paths)

In [17]:
assert len(mask1_paths) == len(mask2_paths)

for mask_idx in range(len(mask1_paths)):
    mask1 = Image.open(mask1_paths[mask_idx])
    mask1 = np.array(mask1)
    mask1 = np.expand_dims(mask1, axis=-1)

    mask2 = Image.open(mask2_paths[mask_idx])
    mask2 = np.array(mask2)
    mask2 = np.expand_dims(mask2, axis=-1)

    assert mask1.shape == mask2.shape

    mask1[mask2 >= 1] = 2
    mask1[mask1 >= 1] = 1
    mask1[mask1 < 1] = 0

    mask_save_path = mask1_paths[mask_idx].replace("RPE-vscans", "multi-vscans-derived")
    mask_save_path = mask_save_path.replace(".png", ".npy")
    if not os.path.exists(os.path.dirname(mask_save_path)):
        os.makedirs(os.path.dirname(mask_save_path)) 
    
    np.save(mask_save_path, mask1)

for image_path in image_paths:
    image = Image.open(image_path)
    image = np.array(image)
    image = np.expand_dims(image, axis=-1)

    image_save_path = image_path.replace("RPE-vscans", "multi-vscans-derived")
    image_save_path = image_save_path.replace(".png", ".npy")
    if not os.path.exists(os.path.dirname(image_save_path)):
        os.makedirs(os.path.dirname(image_save_path)) 
    np.save(image_save_path, image)

In [18]:
train_image_paths = []
val_image_paths = []
train_mask_paths = []
val_mask_paths = []

for image_path in glob.glob(os.path.join("data", "multi-vscans-derived","Original2", "*", "*.npy")):
    if image_path.split(os.path.sep)[3] in ["13", "14", "15"]:
        train_image_paths.append(image_path)
    else:
        val_image_paths.append(image_path)

for mask_path in glob.glob(os.path.join("data", "multi-vscans-derived","Masks_RPE", "*", "*.npy")):
    if mask_path.split(os.path.sep)[3] in ["13", "14", "15"]:
        train_mask_paths.append(mask_path)
    else:
        val_mask_paths.append(mask_path)

data_utils.write_paths(os.path.join("training", "multi-vscans-train-images.txt"), train_image_paths)
data_utils.write_paths(os.path.join("training", "multi-vscans-train-masks.txt"), train_mask_paths)
data_utils.write_paths(os.path.join("validation", "multi-vscans-val-images.txt"), val_image_paths)
data_utils.write_paths(os.path.join("validation", "multi-vscans-val-masks.txt"), val_mask_paths)

In [19]:
all_images = []
for image_path in image_paths:
    image = Image.open(image_path)
    image = np.array(image).astype(np.float32)

    all_images.append(image)
    
all_images = np.stack(all_images, 0)
print(all_images.shape)
print(np.mean(all_images))
print(np.std(all_images))

    

(638, 1024, 400)
47.034603
22.447832
